# Getting Started with Selectools

This interactive notebook walks you through the core concepts of **Selectools** — a Python framework for building AI agents with tool-calling and RAG.

**What you'll learn:**
1. Define tools with `@tool`
2. Create an agent and ask questions
3. Understand the agent loop
4. Add conversation memory
5. Connect to a real LLM
6. Add RAG (document search)

> Steps 1-3 use `LocalProvider` and require **no API key**.

## Step 0: Install

In [ ]:
# Uncomment and run once:
# !pip install selectools

## Step 1: Define a Tool

A **tool** is any Python function decorated with `@tool`. Selectools automatically
generates the JSON schema from your type hints — the LLM never sees your code,
only the schema.

In [ ]:
from selectools import tool


@tool(description="Look up the price of a product")
def get_price(product: str) -> str:
    """Return the price of a product from our catalogue."""
    prices = {"laptop": "$999", "phone": "$699", "headphones": "$149"}
    return prices.get(product.lower(), f"No price found for {product}")


@tool(description="Check if a product is in stock")
def check_stock(product: str) -> str:
    stock = {"laptop": "5 left", "phone": "Out of stock", "headphones": "20 left"}
    return stock.get(product.lower(), f"Unknown product: {product}")


# Inspect the auto-generated schema
print(f"Tool name: {get_price.name}")
print(f"Description: {get_price.description}")
print(f"Parameters: {get_price.parameters}")

## Step 2: Create an Agent

An **Agent** takes your tools and a **Provider** (the LLM backend). We'll use
`LocalProvider` — a built-in stub that works offline. It doesn't call any API;
it simply echoes tool results back, which is perfect for learning the API.

In [ ]:
from selectools import Agent, AgentConfig
from selectools.providers.stubs import LocalProvider

agent = Agent(
    tools=[get_price, check_stock],
    provider=LocalProvider(),
    config=AgentConfig(max_iterations=3),
)

print(f"Agent created with {len(agent.tools)} tools")
print(f"Tool names: {[t.name for t in agent.tools]}")

## Step 3: Ask a Question

`agent.ask()` sends a plain-text prompt, and the agent decides which tool(s) to call.

In [ ]:
result = agent.ask("How much is a laptop?")

print(f"Content:    {result.content}")
print(f"Iterations: {result.iterations}")
print(f"Tool calls: {len(result.tool_calls)}")

if result.tool_calls:
    tc = result.tool_calls[0]
    print(f"\nFirst tool call:")
    print(f"  Tool:   {tc.tool_name}")
    print(f"  Args:   {tc.arguments}")
    print(f"  Result: {tc.result}")

### Anatomy of the Agent Loop

Under the hood, `agent.ask()` runs this loop:

```
1. Build a system prompt that includes your tool schemas
2. Send the prompt + user message to the LLM
3. Parse the response for a TOOL_CALL
4. If found → execute the tool → feed result back to the LLM → repeat from 2
5. If not found → return the response as the final answer
```

The `max_iterations` config caps how many times the loop can repeat.

## Step 4: Multiple Calls and Reset

Each `ask()` call is independent. Use `agent.reset()` to clear internal state.

In [ ]:
# Ask a second question
result2 = agent.ask("Is the phone in stock?")
print(f"Answer: {result2.content}")

# Reset clears accumulated usage stats
agent.reset()
print("\nAgent reset. Ready for a fresh session.")

## Step 5: Connect to a Real LLM

Swap `LocalProvider` for any real provider. Your tools stay exactly the same.

> **Requires** `OPENAI_API_KEY` in your environment. Skip this cell if you
> don't have one — everything above works without it.

In [ ]:
import os

if os.getenv("OPENAI_API_KEY"):
    from selectools import OpenAIProvider
    from selectools.models import OpenAI

    real_agent = Agent(
        tools=[get_price, check_stock],
        provider=OpenAIProvider(default_model=OpenAI.GPT_4O_MINI.id),
        config=AgentConfig(max_iterations=5),
    )

    result = real_agent.ask("Is the laptop in stock and how much is it?")
    print(result.content)
    print(f"\nCost: ${real_agent.total_cost:.6f}")
    print(f"Tokens: {real_agent.total_tokens}")
else:
    print("OPENAI_API_KEY not set — skipping real LLM demo.")
    print("Set it with: %env OPENAI_API_KEY=sk-...")

## Step 6: Add Conversation Memory

`ConversationMemory` keeps track of previous turns so the agent can
reference earlier context.

In [ ]:
from selectools import ConversationMemory

memory = ConversationMemory(max_messages=20)

memory_agent = Agent(
    tools=[get_price, check_stock],
    provider=LocalProvider(),
    config=AgentConfig(max_iterations=3),
    memory=memory,
)

# Turn 1
memory_agent.ask("How much is a laptop?")

# Turn 2 — the agent can reference Turn 1
result = memory_agent.ask("And is that product in stock?")
print(f"Answer: {result.content}")
print(f"Messages in memory: {len(memory)}")

## Step 7: Structured Messages

For advanced use, you can send a list of `Message` objects instead of plain strings.
This is useful for system prompts, multi-role conversations, or injecting context.

In [ ]:
from selectools import Message, Role

messages = [
    Message(role=Role.SYSTEM, content="You are a helpful shopping assistant."),
    Message(role=Role.USER, content="What's the cheapest item you have?"),
]

result = agent.run(messages)
print(result.content)

## Step 8: Add RAG (Document Search)

Give the agent a knowledge base to search. This requires the `rag` extra:

```bash
pip install selectools[rag]
```

> **Requires** `OPENAI_API_KEY` for the embedding provider.

In [ ]:
import os

if os.getenv("OPENAI_API_KEY"):
    try:
        from selectools import OpenAIProvider
        from selectools.embeddings import OpenAIEmbeddingProvider
        from selectools.models import OpenAI
        from selectools.rag import Document, RAGAgent, VectorStore

        embedder = OpenAIEmbeddingProvider(
            model=OpenAI.Embeddings.TEXT_EMBEDDING_3_SMALL.id
        )
        store = VectorStore.create("memory", embedder=embedder)

        docs = [
            Document(
                text="Our return policy allows returns within 30 days.",
                metadata={"source": "policy"},
            ),
            Document(
                text="Shipping takes 3-5 business days for domestic orders.",
                metadata={"source": "shipping"},
            ),
            Document(
                text="Premium members get free expedited shipping.",
                metadata={"source": "membership"},
            ),
        ]

        rag_agent = RAGAgent.from_documents(
            documents=docs,
            provider=OpenAIProvider(default_model=OpenAI.GPT_4O_MINI.id),
            vector_store=store,
        )

        result = rag_agent.ask("How long does shipping take for premium members?")
        print(result.content)
    except ImportError:
        print("RAG extras not installed. Run: pip install selectools[rag]")
else:
    print("OPENAI_API_KEY not set — skipping RAG demo.")
    print("Set it with: %env OPENAI_API_KEY=sk-...")

## What's Next?

You now know the core API! Here's where to go from here:

| Goal | Resource |
|---|---|
| Numbered examples (01–22) | [`examples/`](../examples/) |
| Detailed quickstart guide | [`docs/QUICKSTART.md`](../docs/QUICKSTART.md) |
| Architecture deep-dive | [`docs/ARCHITECTURE.md`](../docs/ARCHITECTURE.md) |
| Tool definition reference | [`docs/modules/TOOLS.md`](../docs/modules/TOOLS.md) |
| Streaming & parallel execution | [`docs/modules/STREAMING.md`](../docs/modules/STREAMING.md) |
| Hybrid search & reranking | [`docs/modules/HYBRID_SEARCH.md`](../docs/modules/HYBRID_SEARCH.md) |
| Full documentation index | [`docs/README.md`](../docs/README.md) |